In [1]:
import numpy as np

# User inputs for seagrass extent
At = float(input("Enter total seagrass extent in the province (ha): "))  # Total seagrass extent
Am = float(input("Enter seagrass extent in the MPA (ha): "))  # Seagrass extent in MPA

# Emission Factors
Et_BAU_percent = float(input("Enter Emission Factor for Baseline (BAU) (%): ")) / 100  # Convert to decimal
Et_MPA_percent = Et_BAU_percent / 2  # Emission factor for MPA is half of BAU (2%)

# Degradation rates
Q_BAU_percent = 2.8 / 100  # Degradation rate for BAU (2.8% per year; Stankovic et al., 2021)
Q_MPA_percent = 1.5 / 100  # Degradation rate for MPA (1.5% per year; Wahyudi et al., 2022)
Q_leakage_percent = 2 / 100  # Leakage degradation rate (2% per year; Wahyudi et al., 2022)

# Seagrass ecological data (user inputs)
biomass = float(input("Enter biomass (g/m²): "))
density = float(input("Enter density (shoots/m²): "))
coverage = float(input("Enter coverage (%): "))

# Compute Standing Carbon Stock (Cstock)
tc_result = np.where(
    (biomass == 0) & (density == 0) & (coverage == 0), 0,
    np.where((biomass > 0) & (density == 0) & (coverage == 0), 10.232724 + 0.297451 * biomass,
    np.where((biomass > 0) & (density == 0) & (coverage > 0), 6.862504 + 0.295653 * biomass + 0.080054 * coverage,
    np.where((biomass > 0) & (density > 0) & (coverage == 0), 7.426061 + 0.298152 * biomass + 0.003718 * density,
    np.where((biomass == 0) & (density == 0) & (coverage > 0), 78.7965 + 0.4117 * coverage,
    np.where((biomass == 0) & (density > 0) & (coverage > 0), 78.7965 + 0.4117 * coverage,
    np.where((biomass == 0) & (density > 0) & (coverage == 0), 96.56068 + 0.001082 * density,
    6.874866 + 0.297085 * biomass + 0.003865 * density + 0.010671 * coverage)))))))

Cstock = tc_result / 100  # Convert g/m² to tC/ha

# ============================
# 1. Calculate Initial Carbon Stocks
# ============================
Stock_total = At * Cstock  # Total carbon stock in the province
Stock_MPA = Am * Cstock  # Total carbon stock in the MPA

# ============================
# 2. Calculate Initial Emissions (Year 0)
# ============================
Q_BAU = Q_BAU_percent * At  # Degradation under BAU
Q_MPA = Q_MPA_percent * Am  # Degradation in MPA
Q_leakage = Q_leakage_percent * Am  # Degradation due to leakage

EF_BAU = Cstock * Et_BAU_percent  # Emission factor for BAU
EF_MPA = Cstock * Et_MPA_percent  # Emission factor in MPA
EF_leakage = Cstock * Et_MPA_percent  # Same as MPA

E_BAU = EF_BAU * Q_BAU  # BAU Emission
E_MPA = EF_MPA * Q_MPA  # Emission inside MPA
E_leakage = EF_leakage * Q_leakage  # Leakage Emission

E_reduced_without_leakage = E_BAU - E_MPA  # Reduction without leakage
E_reduced_with_leakage = E_BAU - (E_MPA + E_leakage)  # Reduction with leakage

# ============================
# 3. Project Emissions for 10 Years
# ============================
years = 10
BAU_emissions = [E_BAU]
MPA_emissions = [E_MPA]
Leakage_emissions = [E_leakage]
Reduced_without_leakage = [E_reduced_without_leakage]
Reduced_with_leakage = [E_reduced_with_leakage]
Percentage_reduction = [((E_BAU-E_reduced_without_leakage) / E_BAU) * 100]  # First year reduction %

for n in range(1, years + 1):
    # Projected BAU emission
    E_BAU_n = Et_BAU_percent * (Stock_total - sum(BAU_emissions)) + BAU_emissions[-1]
    BAU_emissions.append(E_BAU_n)

    # Projected MPA emission
    E_MPA_n = Et_MPA_percent * (Stock_MPA - sum(MPA_emissions)) + MPA_emissions[-1]
    MPA_emissions.append(E_MPA_n)

    # Projected Leakage emission
    E_leakage_n = Et_MPA_percent * (Stock_MPA - sum(Leakage_emissions)) + Leakage_emissions[-1]
    Leakage_emissions.append(E_leakage_n)

    # Projected Reduced Emission
    E_reduced_n_without_leakage = E_BAU_n - E_MPA_n
    Reduced_without_leakage.append(E_reduced_n_without_leakage)

    E_reduced_n_with_leakage = E_BAU_n - (E_MPA_n + E_leakage_n)
    Reduced_with_leakage.append(E_reduced_n_with_leakage)

    # Percentage of emission reduction
    Percentage_reduction.append(((E_BAU_n-E_reduced_n_without_leakage) / E_BAU_n) * 100)

# ============================
# 4. Display Results
# ============================
print("\n--- Segrass Carbon Stock---")
print(f"Standing Carbon Stock (Cstock): {Cstock:.4f} tC/ha")
print(f"Total Carbon Stock (Province): {Stock_total:.2f} tC")
print(f"Total Carbon Stock (MPA): {Stock_MPA:.2f} tC")
print("\n--- Emission Projection Over 10 Years [tC/yr]---")
print(f"{'Year':<5} {'BAU Emission':<15} {'MPA Emission':<15} {'Leakage Emission':<20} {'Reduced E (No Leak)':<20} {'Reduced E (With Leak)':<20} {'% Reduction':<15}")
print("-" * 115)
for n in range(years + 1):
    print(f"{n:<5} {BAU_emissions[n]:<15.2f} {MPA_emissions[n]:<15.2f} {Leakage_emissions[n]:<20.2f} {Reduced_without_leakage[n]:<20.2f} {Reduced_with_leakage[n]:<20.2f} {Percentage_reduction[n]:<15.2f}")



Enter total seagrass extent in the province (ha):  1000
Enter seagrass extent in the MPA (ha):  750
Enter Emission Factor for Baseline (BAU) (%):  4
Enter biomass (g/m²):  50
Enter density (shoots/m²):  50
Enter coverage (%):  50



--- Segrass Carbon Stock---
Standing Carbon Stock (Cstock): 0.2246 tC/ha
Total Carbon Stock (Province): 224.56 tC
Total Carbon Stock (MPA): 168.42 tC

--- Emission Projection Over 10 Years [tC/yr]---
Year  BAU Emission    MPA Emission    Leakage Emission     Reduced E (No Leak)  Reduced E (With Leak) % Reduction    
-------------------------------------------------------------------------------------------------------------------
0     0.25            0.05            0.07                 0.20                 0.13                 20.09          
1     9.22            3.42            3.43                 5.81                 2.37                 37.06          
2     17.83           6.72            6.73                 11.11                4.38                 37.68          
3     25.72           9.88            9.90                 15.84                5.94                 38.42          
4     32.58           12.85           12.86                19.73                6.87             